<a href="https://colab.research.google.com/github/ritwikraha/computer-needs-glasses/blob/master/image-object-tracking/YOLOv8_Object_Region_Counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from ultralytics import YOLO
import cv2
import numpy as np
from google.colab import files
from google.colab.patches import cv2_imshow
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Load YOLOv8 model
model = YOLO('yolov8n.pt')

In [ ]:
# Function to get user input
def get_user_input():
    object_name = input("Enter the object name to count: ")
    x_min = int(input("Enter x_min of the region: "))
    y_min = int(input("Enter y_min of the region: "))
    x_max = int(input("Enter x_max of the region: "))
    y_max = int(input("Enter y_max of the region: "))
    return object_name, (x_min, y_min, x_max, y_max)

# Function to count objects in region
def count_objects_in_region(results, object_name, region):
    count = 0
    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()
        classes = r.boxes.cls.cpu().numpy()

        for box, cls in zip(boxes, classes):
            if (model.names[int(cls)] == object_name and
                box[0] >= region[0] and box[1] >= region[1] and
                box[2] <= region[2] and box[3] <= region[3]):
                count += 1
    return count

In [ ]:
# Main function
def main():
    # Get user input
    object_name, region = get_user_input()

    # Upload image
    uploaded = files.upload()
    img_path = list(uploaded.keys())[0]

    # Read image
    img = cv2.imread(img_path)

    # Run YOLOv8 inference
    results = model(img)

    # Count objects in region
    count = count_objects_in_region(results, object_name, region)

    # Draw region on image
    cv2.rectangle(img, (region[0], region[1]), (region[2], region[3]), (0, 255, 0), 2)

    # Add count text
    cv2.putText(img, f"{object_name} count: {count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display result
    cv2_imshow(img)

# Run the main function
main()